In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\projects\\End-to-End-Movie-Recommendation\\research'

In [2]:
import os

os.chdir('../')
%pwd

'c:\\Users\\44787\\Desktop\\projects\\End-to-End-Movie-Recommendation'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    data_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [11]:
from src.MoviesRecom.constants import CONFIG_PATH
from src.MoviesRecom.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(self, config_file= CONFIG_PATH):
        self.config_path = read_yaml(config_file)
        
        create_directories([self.config_path.artifacts_root])
        
    
    def dataIngestionConfig(self) -> DataIngestionConfig:
        config = self.config_path.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            data_url = config.data_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )   
        
        return data_ingestion_config   

In [16]:
import os
import urllib.request as request
from src.MoviesRecom.logging import logger
from src.MoviesRecom.utils.common import get_size
import zipfile

In [17]:
class DataIngestion:
    def __init__(self, confg: DataIngestionConfig):
        self.config = confg
        
    
    def download_dataset(self):
        
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
                                    url= self.config.data_url,
                                    filename=self.config.local_data_file
                                )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exist, size--> {get_size(Path(self.config.local_data_file))}")
    
    
    def extract_zip(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.dataIngestionConfig()
    data_ingestinion = DataIngestion(confg=data_ingestion_config)
    data_ingestinion.download_dataset()
    data_ingestinion.extract_zip()
except Exception as e:
    raise e

[2024-01-16 16:12:21,817: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-16 16:12:21,822: INFO: common: created directory at: artifacts]
[2024-01-16 16:12:21,825: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-16 16:12:25,692: INFO: 597450681: artifacts/data_ingestion/movie_dataset.zip download! with following info: 
Connection: close
Content-Length: 9525845
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "84494b1348433ffbe767212ee608932b3341153d0f6cc49628120dd6f396d8cf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 20E2:3C1D69:FCA316:10686C1:65A6AAE2
Accept-Ranges: bytes
Date: Tue, 16 Jan 2024 16:12:22 GMT
Via: 1.1 varnish
X-Served-By: cache-lon420102-LON
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1705421542.651987,VS0,VE769
Vary: Authorizat